In [1]:
import pandas as pd
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re
import pdfplumber

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("impact_assessment", self.impact_assessment_node)  # 新增影響評估節點
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "impact_assessment")  # 添加新的邊
        builder.add_edge("impact_assessment", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
            "impact_score": 50,  # 初始化影響程度分數為 50（中等）
        }

    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="以下所有產品皆為基金類型的產品，請依照以下要求列點總結每位客戶的特徵，並在200字內完成。 請確保生成的內容完全基於以下提供的客戶特徵信息，不參考其他來源。格式需對所有客戶保持一致，並使用繁體中文撰寫。 請按以下格式進行列點總結： 1.投資配置：描述總成交量（數值），主要投資產品類型及比例。 2.風險偏好：P值及C值，簡要描述客戶的風險偏好 3.產品偏好：列出客戶偏好的產品類型 4.持有產品風險分佈：RR值分佈。 5.投資策略：概述客戶的主要投資策略。 6.交易狀況：描述近期的交易頻率和活動。 7.持倉狀況：簡述目前的庫存量，包括產品及數值。 8.總結：綜合以上信息進行簡要總結。 本行各種投資型金融商品風險等級由低至高分為四個等級：P1、P2、P3、P4。 P1等級：適合風險承受度為C1（保守型）及以上的客戶。 P2等級：適合風險承受度為C2（穩健型）及以上的客戶。 P3等級：適合風險承受度為C3（成長型）及以上的客戶。 P4等級：適合風險承受度為C4（積極型）客戶。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        state["extracted_features"] = response.content.strip()
        return {"extracted_features": state["extracted_features"]}

    def personalized_news_summary_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="假設你是銀行的理財專員，基於客戶的以下特徵和新聞內容，生成個人化新聞推薦，需符合以下要求： 僅依據以下提供的客戶特徵及新聞內容，不猜測或假設客戶的其他持有產品或喜好。 生成3至6點新聞推薦，最多6點，且影響性最大的新聞需置於最前，不需要說明影響原因。 每點新聞推薦不超過30字，總字數少於300字。 僅根據以下給定的新聞內容和客戶特徵進行分析，避免外部資訊或假設。 請用繁體中文回答。"),
            HumanMessage(content=f"客戶特徵: {state['extracted_features']}"),
            HumanMessage(content=state["news_content"])
        ]
        response = self.model.invoke(messages)
        state["news_summary"] = response.content.strip()
        return {"news_summary": state["news_summary"]}

    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請假設你是銀行的理財專員， 請嚴格基於以下每位客戶特徵和新聞內容，列點生成個人化投資建議，直接給具體建議即可，不要加入其他外部資訊，字數在200字內。投資建議應考慮客戶的風險承受能力、資產配置偏好以及新聞摘要狀況，請用繁體中文回答。"),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        state["investment_advice_result"] = response.content.strip()
        return {"investment_advice_result": state["investment_advice_result"]}

    def impact_assessment_node(self, state: CustomerState):
        messages = [
        SystemMessage(content="你是一位具有 CFA Level 3 資格的資深投資專家，請評估新聞對每個客戶的影響程度。給出0到100分的評分，分數越高代表影響越大。"),
        AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        AIMessage(content=f"新聞摘要: {state['news_summary']}"),
    ]
        response = self.model.invoke(messages)
        impact_text = response.content.strip()

    # 初始化默認值
        score = 20
        explanation = "影響甚小，先給予20分"

    # 使用正則表達式提取数字評分
        score_match = re.search(r'\b(\d{1,3})\b', impact_text)
        if score_match:
            score = int(score_match.group(1))
            score = min(max(score, 0), 100)
            explanation = impact_text[impact_text.find(score_match.group(1)) + len(score_match.group(1)):].strip()
            explanation = explanation.replace('*', '').strip()  # 移除可能的米字號或其他不必要符号

        state["impact_score"] = f"{score} 分"
        return {"impact_score": state["impact_score"]}

    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
            "impact_score": state["impact_score"],  # 包含影響程度分數
        }

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"客戶名稱: {customer_name}\n"
            f"提取的客戶特徵:\n{extracted_features}\n\n"
            f"新聞摘要:\n{news_summary}\n\n"
            f"投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 定義PDF提取摘要的函數
def extract_pdf_summary(pdf_file_path):
    summary = []
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            summary.append(text[:200] + "...")  # 提取前200個字作為摘要
    return "\n".join(summary)

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)
    
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste_3.xlsx")

# 手動輸入新聞內容
news_content_input = """
投資策略:買進健護類股：健護產業的防禦性在美國經濟放緩時受到重視，這從科技股仍在消化上方套牢賣壓，但健護指數已經創下歷史新高就是最好的證明，即使是接下來的降息周期，健護仍可能創造超額報酬。
        買進投資等級金融債：隨著聯準會啟動降息，金融債利差有更大改善空間，以及聯準會提高美國大型銀行的資本要求，為金融市場提供更穩定的環境，並增加金融債的投資信心。
        買進印度股市：印度SENSEX指數已完全收復8月初的跌幅，領先美國標普500指數創下歷史新高，顯示其強勁的經濟表現能抵禦美國經濟放緩所帶來的挑戰，即使在美國降息階段也能延續其好表現。
市場展望:匯市展望：隨著降息成為事實，預期美元指數短線跌深反彈，等待更多經濟數據來確認市場走勢。在G10多數國家同樣進入寬鬆週期下，美元指數仍可能轉強。預期下週美元指數在100~102區間。
        債市展望：聯準會如預期降息兩碼，2026年聯邦基本利率降至3%以下，確立進入降息周期。不過由於市場已部分提前反應，因此預期下週美10年公債殖利率處於3.6~3.8%區間。
        股市展望：半數標普500企業暫停回購股票，高盛統計九月下半月是標普500一年中表現最差的兩周。不過明年標普500企業獲利成長更加均衡，有利第四季表現。預期下週指數在5500~5700區間發展。
頭條新聞:1.聯準會開啟2020年以來首次降息，本輪降息一口氣降息2碼，將基準利率調降至4.75-5.0%，以阻止勞動力市場放緩。除了新冠疫情期間的緊急降息之外，FOMC上一次降息2碼是在2008年全球金融危機期間。
        2.聯準會官員表示，美國監管機構將對銀行資本金改革草案進行全面修改，對大型銀行的影響料比初步方案減少一半，小型銀行可以豁免一大部分措施，近日將公布相關細節。
        3.黎巴嫩發生數百枚呼叫器在不同地點卻幾乎同時間爆炸的恐怖事件，造成數千人傷亡。沒有組織宣稱犯案，不過黎巴嫩武裝組織真主黨指控是以色列所為，揚言報復；以色列未予置評，但說將全面做好攻防準備。
產業訊息:1.微軟*董事會通過一項新的股票回購計畫，規模高達600億美元，這項回購計畫沒有到期日，表示微軟可以持續回購股票，直到600億美元的上限或是決定停止。此外，微軟*也宣布將提高季度股息10%。
        2.分析師郭明錤預估iPhone 16系列從開放預購到首個週末的預購量大約3,700萬支，比iPhone 15衰退12.7%。Apple Intelligence未同步上市以及中國市場競爭激烈，是iPhone 16系列需求疲軟的關鍵原因。
        3.雖然記憶體價格仍在上漲，但摩根士丹利預期DRAM產業將於今年達到周期性高峰，第四季出現反轉，並開始面臨供過於求壓力，接下來訂價環境將更具挑戰，供過於求問題將延續至2026年。
聯準會點陣圖年底前再降兩碼，明年降四碼:1.聯準會經濟預測報告調高今年底失業率到4.4%，雖然明後年也同步調高，但預期勞動市場保持彈性，不會進一步惡化。同時聯準會小幅調降2024年GDP預測值到
            2.0%，明後年則維持先前預測。2.通膨取得進展為聯準會提供更大降息空間。聯準會預估核心PCE物價在2026年底回到目標區間，今年底降至2.6%，明年降至2.2%，聯準會對通膨持續向2%邁進的
             信心增強。3.聯準會本次會議降息兩碼至4.75%~5.0%，預估年底將再降息2碼到4.4%，比六月預估數多降2碼，接近市場定價。預估2025年將再降4碼，2026年再降息2碼，屆時聯邦資金利率將低於3%。
銀行資本改革方案有助提高銀行體質:1.美國聯準會、聯邦存款保險公司以及貨幣監理署曾呼籲將美國銀行和摩根大通等⼋家具有全球系統重要性的美國銀行的資本增加19%，以緩衝意外損失和金融
             衝擊，然最新調高比率僅有9%。2.適度調高最低資本要求，有助於確保銀行在經濟下行時，有足夠的資本來因應潛在損失、較低的調幅可避免銀行減少貸款或是其他風險資產來滿足這個標準，
             進而削減其獲利能力。3.銀行資本改革對金融債有正面的影響，除了更為穩固的銀行體系外，溫和的提高資本要求，能降低對金融系統過度收緊的擔憂，讓金融債保持穩定性。
美國8月零售銷售憂喜參半:1.美國8月零售銷售月增0.1%，優於市場預期的-0.2%，主要受到線上銷售表現以及醫療健保支出強勁的影響。整體零售銷售年增率2.1%雖然比7月的年增2.9%下滑，
             但仍保持穩健。2.由於零售數據優於預期，亞特蘭大聯儲GDPNow模型將第3季實質個人消費支出年增率由3.5%提高至3.7%，實質GDP年增率由2.5%提高至3.0%，有助於美國經濟軟
             著陸。3.不過，扣除汽車和汽油的核心零售銷售月增0.2%，低於預期的0.3%，也比7月0.4%放緩，顯示消費者日常支出的增長較為溫和，而且強調價格透明度或是折扣的線上
             銷售增長更為明顯，顯示美國消費動能仍面臨風險。
本週關注PCE物價與聯準會主席談話:1.7月扣除運輸與國防後的核心耐久財月減0.1%，並且低於預估，顯示企業設備支出疲弱。應留意8月核心耐久財訂單的消長，來解讀企業對未來景氣的看法。預估8
             月耐久財訂單年減2.9%。2.日本通膨壓力升高，導致多位日本央行官員認為應該再次升息。作為觀察日本物價領先指標的東京CPI年增率，在過去四個月當中，有三個月的增幅是擴大的，因此東
             京CPI的走向將牽動日本央行貨幣政策。3.美國7月新屋銷售73.9萬戶，月增10.6%，創下2023年5月以來新高，顯示消費者趁著房貸利率下跌，正在加快進場，同時表現出消費者對未來收入以及經濟穩定的樂
             觀態度，預估8月新屋銷售69.3萬戶。
英國CPI高於預期，中東緊張局勢推高油價:1.英國8月核心CPI年增3.6%，高於7月的3.3%以及分析師預期的3.5%，歐洲央行則是調高今年的通膨預測值，支撐歐元以及英鎊的漲勢。聯準會則表示在朝向2%通膨目
             標上取得進展，美元相對疲弱。2.美元兌日圓一度跌破140的關卡，創下2023年7月以來的最低水準，但隨後因為聯準會降息幅度符合預期、日本首相候選人高市早苗呼籲保持寬鬆，以及日央最新利率
             會議表示不急於升息，因此刺激美元兌日圓跌深反彈。3.黎巴嫩傳呼機爆炸案造成多起傷亡，真主黨將箭頭指向以色列，加劇了中東緊張局勢升高的擔憂，加上墨西哥灣尚有12%產能未能在颶風過後恢復生產，以及降息可
             能刺激需求等預期，激勵上週西德州原油上漲。
多數國家與聯準會同步降息，美元仍能保有優勢:1.自1995年以來聯準會首次降息後三個月，美元兌同步降息國家的匯率平均上漲2.5%，兌非同步降息國家的匯率則是持平0.1%，顯示即使美國進入降息周期，在其他國家同步降息的情境下，美元仍能保有優勢。
             2.過去七次首次啟動降息後三個月，美元兌同步降息國家的匯率表現，普遍優於非同步降息國家的匯率表現，除了1998年長期資本管理公司(LTCM)崩潰所引發美國金融危機以外。
             3.美元指數涵蓋的六種貨幣當中僅有日圓升息，其餘包括歐元、加幣、英鎊、瑞郎以及瑞典克朗等央行都已經開始降息，在多數國家與聯準會同步降息的情況下，預期美元指數在首次降息後三個月仍能保有優勢。
期貨、實物ETF與全球央行持續同步作多黃金:1.CFTC黃金期貨的淨多單仍維持在2022年以來的高點附近，顯示市場對黃金的投機需求仍然相當強勁，看漲黃金的情緒增強。
             2.世界黃金協會最新統計，8月全球實物黃金ETF淨流入21億美元，是連續第四個月淨流入，來自北美和歐洲的資金推高了黃金的總持倉量，全球經濟的不確定性上升是推動資金流入的原因。
             3.全球央行7月增持37公噸黃金，是一月以來的最高購買量，顯示央行對黃金的購買熱潮仍在持續。波蘭央行與印度央行並列第二季最大的黃金買家，波蘭央行的目標是至少持有20%的國家儲備黃金，使波蘭成為可
             信任的國家，目前持有14.7%。  
公債反映降息符合預期，利差則明顯收窄:1.聯準會啟動2020年以來首次降息，但由於市場已提前反映，加上美國整體零售銷售數據優於預期，因此美國公債指數反倒有些買預期賣事實的賣壓出籠，上週回檔是
             近三周來首見。2.美國公司債利差取得重大進展 。投資級債指數利差較上周收窄5.3個基點，降幅5.5%，非投資級債指數利差收窄20.8個基點，降幅6.4%，反映企業體質可望受惠美國進
             入降息周期。3.中國經濟數據疲軟推升人行降息降準的預期、油價反彈提高中東與拉美的持債信心以及美元貶值提高非美貨幣資產價值等多重因素激勵，新興市場債指數普遍上漲。
首次降息後六個月，利率仍可能持續下滑:1.美國1984年以來經歷10次降息週期，高盛將其中3次是歸類為預防性降息(經濟相對穩定，但在經濟放緩前降息)、4次為衰退性降息(衰退或即將來臨)、以及3次
             利率正常化降息(經濟已經放緩，但避免陷入衰退)。2.三種情境下，兩年期公債殖利率在首次降息前六個月的表現都出現下跌，並且延續到首次降息後六個月，不過在成長恐慌性降息以及利率正常化降息的情境下，
             跌幅較小。3.隨著經濟發展情勢更為明朗，除非是衰退性降息，否則六個月後將開始反彈。
銀行業的利差還有收窄空間:1.聯準會升息、信用風險上升或是銀行監管收緊時，通常會推升銀行業利差擴大，因為銀行持有債券資產價值會因為升息而縮水、信用風險上升會提高銀行貸款
             損失準備，加強監管會提高銀行營運成本。2.銀行投等債與美國投等債的相對利差在2023年達到高點，但隨著通膨數據放緩，銀行相對非金融業利差開始收窄，但仍高於2013~2022年多數時間在0.8~0.9
             的發展區間。3.聯準會最新利率點陣圖顯示美國未來仍有進一步降息空間，以及銀行資本改革僅適度加強監管(非激進)，可望提供銀行業相對利差進一步下滑的可能性。
標普500再創歷史新高，新興市場受惠美元貶值:1.由於聯準會提出符合市場預期的降息路徑，經濟可能陷入衰退的疑慮消退，激勵標普500再度創下歷史新高，收復7月中旬套利交易平倉前所創下的高點，但隨後三巫日
             的平倉賣壓以及聯邦快遞獲利低於預期，指數高檔震盪。2.美元貶值是推動MSCI新興市場指數走高的主要原因。除印度股市因為經濟表現強勁創下歷史新高外，中國經濟
             數據疲弱、巴西升息壓力浮現以及墨西哥通過具有爭議的司法改革等，多數當地指數表現持平。3.標普500能源指數第二季獲利與去年同期持平，分析師預估較去年第三季獲利衰退約17%，營運尚未出現轉機，
             研判能源股近期漲勢主要是隨著油價跌深反彈。
標普500企業盈餘明年成長更加均衡:1.高成長帶來高股價。2023年標普500中7大巨頭獲利成長35%，遠優於其他493家企業衰退4%，推動股市資金向7大巨頭靠攏，2023年7大巨頭股價累積上漲78%，
             超越其他493家的漲幅12%。2.2024年7大巨頭預估獲利增長29%，再度優於其他493家企業預估獲利成長3%，所以今年前8個月7大巨頭上漲32%，也超越493家企業的漲幅14%。
             3.由於企業獲利成長加速有利於股價上漲，因此明、後年這493家標普500企業獲利成長加速，分別達到14%與10%，將有利標普500未來的漲勢更加均衡。
健護類股在首次降息後有超額報酬:1.統計自1995年以來，標普500健護指數在聯準會首次降息前1個月表現，落後標普500指數報酬率1.2%，但是在降息後3個月，走勢則轉趨強勁，相較標普500出
             現3.6%的超額報酬。2.醫療保險產業在健護指數次產業當中表現最為強勁，在首次降息後三個月，相較標普500有6.4%的超額報酬，顯示大型醫療保險類股除了在經濟放緩周期能發
             揮防禦特質外，也能受惠寬鬆貨幣政策。3.大型生技公司通常在首次降息前一個月，就能超越標普500指數，並且能延續到降息後三個月的次產業，這反映生技公司在進入寬鬆週期後，能以更優惠的條件
             取得資金以進行長期研發。
印度股市在非衰退降息期間表現亮眼:1.MSCI印度在聯準會降息後的表現，無論是在非衰退期間還是所有情境下，市場在降息後1個月或3個月對降息的反應都呈現正報酬，顯示MSCI印度對於美聯儲降
             息後的短期市場反應較為積極。2.在非衰退期間降息的情境下，6個月後的中位數報酬率高於3個月的表現，顯示降息與經濟增長的加乘效果，有助於對市場取得更好的表現，但是在衰退的情境下，
             報酬則是隨著時間遞減。3.MSCI印度在非衰退情境下，首次降息6個月後的報酬率高於7%，明顯高於所有情境下的負報酬，顯示市場在經歷短期降息的利多激勵之後，仍將以經濟表現為
             投資依據。
機會市場亮點:美國聯準會啟動降息循環，標普500企業明年獲利成長將更加均衡，第四季行情值得期待。中國第四季可能再度降準以提振中國經濟，寬鬆貨幣環境與刺激政策有利陸股。
            印度領先標普500創歷史新高，顯示其強勁經濟表現能抵禦美國經濟放緩的挑戰。AI與高速運算需求暢旺、時序進入歐美消費旺季，均有利台灣出口保持暢旺。
            AI新晶片供不應求，邊緣AI以及網路設備需求持續加溫，科技業獲利持續成長。銀行資本改革的不確定性消失，第四季是金融業股票回購高峰，均有利股價動能。
            健護類股除了迎來獲利增長加速以外，防禦特性可望在經濟放緩時受到重視。聯準會開始降息，並且提出明確降息路徑，有利公債走高。大型銀行將提高資本要求，有助於金融環境更為穩定，並提高金融債投資信心。
            ECB再次降息並指經濟傾向下行，配合盤前幾位ECB官員釋出談話偏鴿，寬鬆預期再升溫，基本面壓力不致在近期發生，持續的政策寬鬆，推升歐元區景氣仍需時間發酵。
            日央認為目前金融市場仍不穩定，將繼續密切監控市場情勢，升息並非迫在眉睫。美國降息，日圓兌美元仍有升值可能，日股將因日圓的波動而持續震盪。
            美國消費動能走緩，8月零售數據也顯示民眾更傾向利用電商進行消費。所幸情況尚不至於太過悲觀，業者相繼推出低價方案因應，維持全年業績目標不變。
            美國製造業ISM已落入底部區域，加上過去2年美國製造業的大型投資計劃明顯上升，預期2025年有望迎來復甦，惟短線將受航太軍工族群估值過高的拖累。
            美國經濟動能放緩以及夏季用油旺季結束，油價可能陷入整理，能源股暫時缺乏利多。公用事業股受益於美債殖利率下滑，但市場對於聯準會的降息預期已高，公用事業可逢高停利。
            聯準會如市場預期降息兩碼，美國公債殖利率可能利多出盡，加上9~10月是美國求職旺季，就業數據可能偏強，不排除殖利率向上彈升，不利REITs走勢。
            聯準會降息後，非投資等級債利差明顯收斂，市場預期降息有助於改善企業體質。但仍建議避開CCC級債有基本面惡化的疑慮，相對靠攏評級較高的BB級債。
            新興美元債有望受惠聯準會降息題材，區域中短線相對看好亞太區信用債，具備收益優勢、政治穩定及經濟動能良好；拉丁美洲留意墨西哥司法改革爭議延燒。
"""

# 定義輸入的新聞內容
state = {
    "news_content": news_content_input
}

# 生成客戶特徵
all_customers_output = []
all_customers_data = []

# 自動讀取Excel欄位名稱，排除ID欄位
selected_columns = df.columns[1:]

# 迭代每個客戶，生成特徵摘要和投資建議
for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 初始化每位客戶的狀態，確保不受之前狀態的影響
    state = {
        "news_content": news_content_input,  # 保持新聞內容不變
        "customer_features": [],  # 清空之前的客戶特徵
        "extracted_features": "",
        "news_summary": "",
        "investment_advice_result": "",
        "impact_score": 50  # 初始化影響程度分數為 50（中等）
    }
    
    # 根據選擇的欄位來生成客戶特徵
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 更新狀態的客戶特徵
    state["customer_features"] = customer_features

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 確保投資建議中只包含投資建議的結果，而非新聞摘要
    # 你可以檢查並確保 state["investment_advice_result"] 僅包含投資建議
    if result["news_summary"] in result["investment_advice_result"]:
        result["investment_advice_result"] = result["investment_advice_result"].replace(result["news_summary"], "").strip()

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        result["extracted_features"], 
        result["news_summary"],  # 保留新聞摘要
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵摘要": result["extracted_features"],
        "新聞摘要": result["news_summary"],  # 保留新聞摘要
        "投資建議": result["investment_advice_result"],  # 只保留投資建議
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output_V01.1_1.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\senpai-m2n-YTg9-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gemini produced an empty response. Continuing with empty message
Feedback: 


警告: ID14382 的投資建議生成為空！
Excel 文件已生成並保存，請檢查結果。
